## LADOT one trip - projecting distance for all vp

In [ ]:
import branca 
import geopandas as gpd
import numpy as np
import pandas as pd

import create_table
import utils
from update_vars import analysis_date, PROJECT_CRS

In [ ]:
trips = create_table.get_table(
    "trips", 
    analysis_date, 
    filters = [[("name", "==", "LA DOT Schedule")]]
)

In [ ]:
shapes = create_table.get_table(
    "shapes", 
    analysis_date, 
    columns = ["shape_array_key", "shape_id", "n_trips", "geometry"],
    filters = [[
        ("shape_array_key", "in", trips.shape_array_key.unique().tolist())]]
)

In [ ]:
shapes.explore("shape_id", tiles="CartoDB Positron")

In [ ]:
stops_projected = create_table.stop_times_projected_table(
    analysis_date, 
    filters = [[("trip_id", "in", trips.trip_id.unique().tolist())]],
)

In [ ]:
trip_cols = ["schedule_gtfs_dataset_key", "trip_id", "shape_id"]

check_df = (stops_projected
                   .sort_values(trip_cols + ["stop_sequence"])
                   .groupby(trip_cols)
                   .agg({"stop_meters": lambda x: list(x)})
                   .reset_index()
                  )

check_df = check_df.assign(
    is_monotonic = check_df.apply(lambda x: np.all(np.diff(x.stop_meters) > 0), axis=1)
)

check_df.is_monotonic.value_counts() 

In [ ]:
loopy_shapes = check_df[check_df.is_monotonic == False].shape_id.unique()
ok_shapes = check_df[check_df.is_monotonic == True].shape_id.unique()

print(f"# loopy shapes: {len(loopy_shapes)}, # ok shapes: {len(ok_shapes)}")

In [ ]:
shapes[shapes.shape_id.isin(loopy_shapes)].explore("shape_id", tiles="CartoDB Positron")

In [ ]:
shapes[shapes.shape_id.isin(ok_shapes)].explore("shape_id", tiles="CartoDB Positron")

In [ ]:
one_trip = "30-n30kvaejc"

trip_filter = [[("trip_id", "==", one_trip)]]

trips = create_table.get_table(
    "trips", 
    analysis_date, 
    filters = trip_filter
)

trips

## Look at what's in all the tables

In [ ]:
stop_times_direction = create_table.get_table(
    "stop_times_direction",
    analysis_date,
    filters = trip_filter,
    columns = ["trip_id", "stop_sequence", "geometry"]
).to_crs(PROJECT_CRS)


shapes = create_table.get_table(
    "shapes",
    analysis_date,
    filters = [[("shape_id", "in", trips.shape_id)]],
    columns = ["shape_id", "geometry"]
).to_crs(PROJECT_CRS)

In [ ]:
vp = create_table.get_table(
    "vp", 
    analysis_date,
    filters = trip_filter,
    columns = [
        "trip_id", 
        "location_timestamp_local", "geometry"
    ]
).to_crs(PROJECT_CRS).sort_values(
    "location_timestamp_local"
).reset_index(drop=True)

vp.shape

In [ ]:
vp.head()

In [ ]:
m = utils.plot_vp_shape_stops(
    vp,
    shapes,
    stop_times_direction,
    vp_as_line=True
)

m

In [ ]:
m2 = utils.plot_vp_shape_stops(
    vp,
    shapes,
    stop_times_direction,
    vp_as_line=False
)

m2

## Put stop_times, trips, stops, shapes tables together

In [ ]:
stops_projected = create_table.stop_times_projected_table(
    analysis_date, 
    filters = trip_filter,
)

# We don't do this in our pipeline, because vp meters is an array
# But this illustrates the point more clearly
stops_projected = stops_projected.assign(
    subseq_stop_meters = stops_projected.groupby(["schedule_gtfs_dataset_key", "trip_id"]).stop_meters.shift(-1)
)

In [ ]:
stops_projected.head()

## Put vp with shape

In [ ]:
vp_projected = create_table.vp_projected_table(
    analysis_date,
    filters = trip_filter
)   

In [ ]:
vp_projected.head()

## What do we need to calculate speed between stop 5 and 6?

Find the speed between stop_sequence 5 and 6.

* For each stop pair (here, defined by `stop_seq_pair`), we grab a subset of the vp that fall between, and calculate speed.
* They can be thought about as groups along a number line. As the bus travels from one stop to the next, there are a grouping of vp that can be used to derive speed, on, and on, and for each trip, we want to repeat it programatically, without knowing the nuances of the start/end numbers of each group.
* The first improvement on this is to actually capture the relevant before/after vp for each stop. Ex: if the bus is seen 10 feet before the stop and also 5 feet after the stop, presumably the bus arrives at the stop (0 feet) somewhere in between those 2 timestamps. 
   * We want to calculate an estimated RT stop arrival because that will help us compare scheduled stop times with RT stop times.

In [ ]:
def stops_and_vp_between_two_stops(
    stop_pair: str,
) -> tuple[pd.DataFrame]:
    """
    """
    stops_projected_subset = stops_projected.loc[
        (stops_projected.stop_seq_pair == stop_pair)
    ]
    
    vp_projected_subset = vp_projected.loc[
        (vp_projected.vp_meters >= stops_projected_subset.stop_meters.min()) & 
        (vp_projected.vp_meters <= stops_projected_subset.stop_meters.max())
    ]

    return stops_projected_subset, vp_projected_subset

def calculate_time_elapsed(
    df: pd.DataFrame,
    timestamp_col: str = "location_timestamp_local"
) -> float:

    seconds_elapsed = (df[timestamp_col].max() - df[timestamp_col].min()) / (np.timedelta64(1, "s"))
    
    return seconds_elapsed
    
    
def calculate_meters_elapsed(
    df: pd.DataFrame,
    distance_col: str = "vp_meters"
) -> float:
    meters_elapsed = df[distance_col].max() - df[distance_col].min()

    return meters_elapsed


def print_speed_components(
    df, 
    timestamp_col: str = "location_timestamp_local", 
    distance_col = "vp_meters"
):
    sec_elapsed = calculate_time_elapsed(df, timestamp_col)
    meters_elapsed = calculate_meters_elapsed(df, vp_meters)
    print(f"seconds: {sec_elapsed}, meters: {meters_elapsed}")

    speed = utils.calculate_speed(meters_elapsed, sec_elapsed)
    print(f"speed for segment: {speed}")

In [ ]:
stops_subset, vp_subset = stops_and_vp_between_two_stops(5, 6)

In [ ]:
stops_subset

In [ ]:
# Same info presented
stops_subset[stops_subset.stop_seq_pair=="5__6"]

In [ ]:
vp_subset

In [ ]:
m3 = utils.plot_vp_shape_stops(
    vp_projected[vp_projected.vp_idx.isin(vp_subset.vp_idx)],
    shapes,
    stops_projected[
        (stops_projected.stop_seq_pair=="5__6")],
    vp_as_line=False
)

m3

In [ ]:
print_speed_components(vp_subset)

## What do we need to calculate speed between stop 49 and 50?

Find the speed between stop_sequence 49 and 50.

In [ ]:
stops_subset2, vp_subset2 = stops_and_vp_between_two_stops(49, 50)
stops_subset2

In [ ]:
stops_subset2

In [ ]:
m4 = utils.plot_vp_shape_stops(
    vp_projected[vp_projected.vp_idx.isin(vp_subset2.vp_idx)], 
    shapes,
    stops_projected[
        (stops_projected.stop_seq_pair=="49__50")],
    vp_as_line=False
)

m4

In [ ]:
print_speed_components(vp_subset2)

## Add speed

Demo the method:
* we use the same vp for each trip, so let's turn that into arrays
* for each segment, we have a begin and end stop_meters
* look within the array to find which vp observations we need to calculate the change in time and change in distance over
* calculate speed

Overall, we look through every vp, and calculate the speeds between stops.
It's imperfect now, because what if there aren't rows over which to calculate our change?
Actually, we use 2 vehicle positions, before and after a stop, and then estimate the arrival time at the stop, and then calculate that change.
But given that this trip has about 50 stops and 200 vp, we'd basically be using 100 of those vp, some of them redundant too.

In [ ]:
vp_meters_arr = vp_projected.vp_meters.to_numpy()
vp_time_arr = vp_projected.location_timestamp_local.to_numpy()


speed_series = []

for row in stops_projected.itertuples():
    begin_stop_meters = getattr(row, "stop_meters")
    end_stop_meters = getattr(row, "subseq_stop_meters")
    
    grab_indices = np.where(
        (vp_meters_arr >= begin_stop_meters) & 
        (vp_meters_arr <= end_stop_meters)
    )[0]
    
    if grab_indices.size > 0:
        distance_array = vp_meters_arr[grab_indices]
        time_array = vp_time_arr[grab_indices]

        speed = utils.calculate_speed(
            distance_array.max() - distance_array.min(), 
            (time_array.max() - time_array.min()) / np.timedelta64(1, "s")
        )

        speed_series.append(speed)
    else:
        speed_series.append(np.nan)


In [ ]:
speed_df = stops_projected.assign(
    speed_mph = speed_series
)

In [ ]:
speed_df.speed_mph.min(), speed_df.speed_mph.max()

In [ ]:
ACCESS_ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdBu_10.scale(vmin=0, vmax=30)
speed_df[
    speed_df.speed_mph.notna()
].explore(
    "speed_mph", 
    cmap=ACCESS_ZERO_THIRTY_COLORSCALE,
    tiles = "CartoDB Positron"
)

## Methodology
* Project each stop position and vehicle position onto shape
* Use that to find distance and time elapsed
* Speed can be calculated between stops

### Real World Complexities
* This one trip, for the most part, it's not that many vp between each stop. Are we saving much time if we filter it out anyway?
   * It depends on the framework of what you use to define a segment.
   * A stop-to-stop segment is fairly finite. But what if we move to corridors? Each city block?
   * If we start combining trips across multiple operators that travel along the same street, how do we go about filtering efficiently without calculating every delta there is, and using only a fraction of those to calculate what we're interested in?
* Ideally, the meters progressed increases monotonically, though that's not true for about 1/3 of the routes where there is loop or inlining occurring. If a bus double backs along any portion of the shape, (going one way along a major street, then back along it; exiting a plaza), then `vp_meters` can actually decrease for a bit without being incorrect.
   * We need an additional data processing step...why `stop_times_direction` was created, we want to know what a stop's primary direction of travel is.
   * We should add something similar to vp.
   * If a vp isn't moving, the `vp_primary_direction="Unknown"`, and actually that helps us get at dwell times too.
   * This is not a dwell time at a stop necessarily, but how many vp observations did we capture without the bus moving (aka traffic). 
   * For a single day, for all operators with RT, this narrows down the rows from 15M to 12M (so that's a nice chunk that we can roll-up!)
   * Nearest neighbors will help us get at this, because we can move to a more stop-agnostic framework. While transit will use stop-to-stop segments, maybe we can also ask what are speeds for transit that travel along a corridor, and use each city block as our segment.
   * The less granular we go, the more we can quickly filter through to find the vp we're most interested in.
   * The more granular we go, eventually we do converge at calculating all the deltas.